In [1]:
# pip install pyspark
# pip install duckdb

In [2]:
import duckdb
from pyspark.sql import SparkSession

In [3]:


# Inicializar una sesión de Spark
spark = SparkSession.builder \
    .appName("ParquetToDuckDB") \
    .config("spark.jars.packages", "im.duck:duckdb:0.10.1") \
    .getOrCreate()

# Leer datos desde un archivo Parquet
df = spark.read.parquet("data_lake/compravenda_sup.parquet")

# Mostrar el esquema de los datos
df.printSchema()

# Mostrar las primeras filas de los datos
df.show()

df.write.format("duckdb").option("database", "nombre_de_tu_base_de_datos").option("table", "nombre_de_tu_tabla").mode("overwrite").save()


# Una vez que hayas terminado de trabajar con los datos, es buena práctica cerrar la sesión de Spark
spark.stop()


24/03/22 12:32:40 WARN Utils: Your hostname, Ruths-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.192.24.59 instead (on interface en0)
24/03/22 12:32:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ruthparajo/.ivy2/cache
The jars for the packages stored in: /Users/ruthparajo/.ivy2/jars
im.duck#duckdb added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a8bfbed-3646-4f2f-9996-2d17b2d08fa2;1.0
	confs: [default]
:: resolution report :: resolve 2881ms :: artifacts dl 0ms
	:: modules in use:
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   0   |   0   |
	---------------------------------------------------------------------

:: problems summary ::
:::: WARNINGS
		module not found: im.duck#duckdb;0.10.1

	==== local-m2-cache: tried

	  file:/Users/ruthparajo/.m2/repository/im/duck/duckdb/0.10.1/duckdb-0.10.1.pom

	  -- artifact im.d

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.